In [ ]:
#In the below code, we first clean the dataset by changing all numeric continuous attributes into categorical ones. As far as the capital gain and capital loss attributes
#are concerned, I find the median of all non-zero values and then categorise these values into low and high
#Everything else is a code representing the apriori algorithm.
#The data structures used are:
#set
#list
#numpy array
#dictionary


In [11]:
import pandas as pd
import timeit
from numpy import *
start = timeit.default_timer()
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None
                , na_values = "?")
df.columns = ['age', 'workclass','fnlwgt','education no', 'education', 'marital status', 'occupation', 'relationship', 'race',
              'sex', 'capital gain', 'capital loss','hours per week','native country', 'class']
print df.head()
df.drop(['education no', 'fnlwgt'], axis = 1)
def transformAge(d1):
    if d1 <= 25: return 'Young'
    elif 26 < d1 <= 45 : return 'Middle Aged'
    elif 46<= d1 <= 65 : return 'Senior'
    else : return 'Old'
df["Age"]= df['age'].map(transformAge)    

def transformHours(d1):
    if d1 <= 25: return 'Part time'
    elif 25 < d1 <= 40 : return 'Full time'
    elif 40 < d1 <= 60 : return 'Overtime'
    else: return 'Too much'
df["Hours per week"]= df['hours per week'].map(transformHours)
df.drop(['hours per week'], axis = 1)
import numpy as np
capital = []
for index, row in df.iterrows():
    if (row['capital loss'] != 0):
        capital.append(row['capital loss'])
array1 = np.array(capital)
print np.median(array1)
capital2 = []
for index, row in df.iterrows():
    if (row['capital gain'] != 0):
        capital2.append(row['capital gain'])
array2 = np.array(capital2)
print np.median(array2)

def transformCapitalloss(d1):
    if (d1 == 0): return 'nil'
    elif (d1 <= 1887): return 'Low'
    else : return 'High'
df['Capital loss'] = df['capital loss'].map(transformCapitalloss)

def transformCapitalgain(d1):
    if (d1 == 0): return 'nil'
    elif (d1 <= 7298): return 'Low'
    else : return 'High'
df['Capital gain'] = df['capital gain'].map(transformCapitalloss)

df_cleaned = df.drop(['age','fnlwgt','education','hours per week','capital loss', 'capital gain'], axis = 1)
print df_cleaned.head()  
def loadDataSet():
    return df_cleaned.values.tolist()
   
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))
def scanDatabase(D, Ck, minSupport):
    supportcount = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in supportcount: supportcount[can]=1
                else: supportcount[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in supportcount:
        support = supportcount[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData
dataSet = loadDataSet()
C1 = createC1(dataSet)
D = list(map(set,dataSet))
L1,suppDat0 = scanDatabase(D,C1,0.5)
L1
L1 = list(map(set, L1))
L1
def aprioriGen(Lk, k): 
    candidates = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: 
                candidates.append(Lk[i] | Lk[j]) 
    return candidates
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanDatabase(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanDatabase(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData
L,suppData = apriori(dataSet)
frequent_itemsets = list(map(set, L))
print frequent_itemsets
stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time) + "seconds")


   age          workclass  fnlwgt education no  education  \
0   39          State-gov   77516    Bachelors         13   
1   50   Self-emp-not-inc   83311    Bachelors         13   
2   38            Private  215646      HS-grad          9   
3   53            Private  234721         11th          7   
4   28            Private  338409    Bachelors         13   

        marital status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital gain  capital loss  hours per week  native country   class  
0          2174             0              40   United-States   <=50K  
1             